#### Faiss
Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning.

In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader=TextLoader("speech.txt")
documents=loader.load()
text_splitter=CharacterTextSplitter(chunk_size=1000,chunk_overlap=50)
docs=text_splitter.split_documents(documents)


In [2]:
docs

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\nâ€¦'),
 Document(metadata={'source': 'speech.txt'}, page_content='â€¦\n\nIt will be all the easier for us to conduct

In [4]:
embeddings=OllamaEmbeddings(model="mxbai-embed-large")
db=FAISS.from_documents(docs,embeddings)
db

In [5]:
### querying 
query="We desire no conquest"
docs=db.similarity_search(query)
docs


[Document(id='e33ad51c-5a0e-45f1-b0c4-7663d7b15e80', metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\nâ€¦'),
 Document(id='08a1c446-6ae5-4ca4-a70f-c2c54ae8eb91', metadata={'so

#### As a Retriever
We can also convert the vectorstore into a Retriever class. This allows us to easily use it in other LangChain methods, which largely work with retrievers

In [8]:
retriever=db.as_retriever()
docs=retriever.invoke(query)
docs[0].page_content

'The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\nâ€¦'

#### Similarity Search with score
There are some FAISS specific methods. One of them is similarity_search_with_score, which allows you to return not only the documents but also the distance score of the query to them. The returned distance score is L2 distance. Therefore, a lower score is better.

In [9]:
docs_and_score=db.similarity_search_with_score(query)
docs_and_score

[(Document(id='e33ad51c-5a0e-45f1-b0c4-7663d7b15e80', metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\nâ€¦'),
  np.float32(252.9653)),
 (Document(id='08a1c446-6ae5-4ca4-a70f-c

In [10]:
embedding_vector=embeddings.embed_query(query)
embedding_vector

[0.27478092908859253,
 0.3209671676158905,
 -0.3462097942829132,
 0.4715955853462219,
 -0.00568079948425293,
 -0.4666594862937927,
 -0.40182679891586304,
 1.066915512084961,
 -0.1142488345503807,
 -0.3130856156349182,
 0.26537612080574036,
 0.19278906285762787,
 -0.4075538218021393,
 0.2648811340332031,
 -0.8756508231163025,
 -0.38360193371772766,
 -1.0933624505996704,
 0.4347361922264099,
 -0.17886857688426971,
 0.2576620876789093,
 -0.6985749006271362,
 -0.06202217563986778,
 -0.5518927574157715,
 -0.32350942492485046,
 -0.41837117075920105,
 0.009081313386559486,
 -0.3976304829120636,
 0.7224384546279907,
 1.2528622150421143,
 1.3024728298187256,
 -0.09633568674325943,
 0.042150646448135376,
 -0.7057452201843262,
 -0.0049410052597522736,
 -0.2321907877922058,
 -0.4907412827014923,
 0.7156764268875122,
 -0.7808824181556702,
 -0.2026333212852478,
 -0.1992907077074051,
 -0.6014580726623535,
 -0.015915118157863617,
 0.6268371939659119,
 0.15766501426696777,
 -0.8800399899482727,
 -0.802

In [11]:
docs_score=db.similarity_search_by_vector(embedding_vector)
docs_score

[Document(id='e33ad51c-5a0e-45f1-b0c4-7663d7b15e80', metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\nâ€¦'),
 Document(id='08a1c446-6ae5-4ca4-a70f-c2c54ae8eb91', metadata={'so

In [12]:
### Saving And Loading
db.save_local("faiss_index")

In [13]:
new_db=FAISS.load_local("faiss_index",embeddings,allow_dangerous_deserialization=True)
docs=new_db.similarity_search(query)

In [14]:
docs

[Document(id='e33ad51c-5a0e-45f1-b0c4-7663d7b15e80', metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\nâ€¦'),
 Document(id='08a1c446-6ae5-4ca4-a70f-c2c54ae8eb91', metadata={'so